# Решение системы линейных алгебраических уравнений

## Постановка задачи

Решить СЛАУ при **n = 12**:

$$\begin{cases}
a_{11}x_1 + a_{12}x_2 + \ldots + a_{1n}x_n = f_1 \\
a_{21}x_1 + a_{22}x_2 + \ldots + a_{2n}x_n = f_2 \\
\vdots \\
a_{n1}x_1 + a_{n2}x_2 + \ldots + a_{nn}x_n = f_n
\end{cases}$$

где:
- $a_{ii} = 1$
- $a_{ij} = \frac{1}{i^2 + j}$ при $i \neq j$
- $f_i = \frac{1}{i}$

---

## Методы решения:

### Прямые методы:
1. **Метод Гаусса с выбором главного элемента**
2. **Метод LU-разложения**

### Итерационные методы:

**Классические:**

3. **Метод Якоби**

4. **Метод Зейделя**

5. **Метод верхней релаксации (SOR)**

**Градиентные:**

6. **Метод градиентного спуска**

7. **Метод минимальных невязок**

8. **Метод сопряжённых градиентов**

9. **Стабилизированный метод бисопряжённых градиентов (BiCGSTAB)**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List
import pandas as pd

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
np.set_printoptions(precision=6, suppress=True)

# Создание системы
def create_system(n: int) -> Tuple[np.ndarray, np.ndarray]:
    """Создаёт систему линейных уравнений согласно заданию"""
    A = np.zeros((n, n))
    f = np.zeros(n)
    
    for i in range(n):
        for j in range(n):
            if i == j:
                A[i, j] = 1.0
            else:
                A[i, j] = 1.0 / ((i + 1)**2 + (j + 1))
    
    for i in range(n):
        f[i] = 1.0 / (i + 1)
    
    return A, f

n = 12
A, f = create_system(n)

print(f'Размерность системы: {n}x{n}')
print(f'Определитель матрицы A: {np.linalg.det(A):.6e}')
print(f'Число обусловленности: {np.linalg.cond(A):.6e}')
print('\nМатрица A:')
print(A)
print('\nВектор f:')
print(f)

---
# ПРЯМЫЕ МЕТОДЫ

Прямые методы позволяют найти точное решение системы за конечное число операций .

## 1. Метод Гаусса с выбором главного элемента

### Описание метода:

Метод Гаусса состоит из двух этапов:

**1. Прямой ход** - приведение матрицы к верхнетреугольному виду:
- На каждом шаге $k$ выбирается главный элемент (максимальный по модулю в столбце)
- Строки переставляются для устойчивости
- Исключаются элементы ниже диагонали:

$$a_{ij}^{(k+1)} = a_{ij}^{(k)} - \frac{a_{ik}^{(k)}}{a_{kk}^{(k)}} a_{kj}^{(k)}, \quad i > k$$

**2. Обратный ход** - вычисление решения из верхнетреугольной системы:

$$x_i = \frac{1}{a_{ii}}\left(f_i - \sum_{j=i+1}^{n} a_{ij}x_j\right), \quad i = n, n-1, \ldots, 1$$

### Сложность: $O(n^3)$

In [ ]:
def gauss_with_pivot(A: np.ndarray, f: np.ndarray) -> np.ndarray:
    """Решение СЛАУ методом Гаусса с выбором главного элемента"""
    n = len(f)
    A_work = A.copy()
    f_work = f.copy()
    
    # Прямой ход
    for k in range(n - 1):
        # Поиск главного элемента в столбце k
        max_row = k
        for i in range(k + 1, n):
            if abs(A_work[i, k]) > abs(A_work[max_row, k]):
                max_row = i
        
        # Перестановка строк
        if max_row != k:
            A_work[[k, max_row]] = A_work[[max_row, k]]
            f_work[k], f_work[max_row] = f_work[max_row], f_work[k]
        
        # Исключение переменных
        for i in range(k + 1, n):
            if A_work[k, k] != 0:
                factor = A_work[i, k] / A_work[k, k]
                for j in range(k, n):
                    A_work[i, j] -= factor * A_work[k, j]
                f_work[i] -= factor * f_work[k]
    
    # Обратный ход
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        sum_val = 0.0
        for j in range(i + 1, n):
            sum_val += A_work[i, j] * x[j]
        x[i] = (f_work[i] - sum_val) / A_work[i, i]
    
    return x

# Решение
x_gauss = gauss_with_pivot(A, f)
residual_gauss = np.linalg.norm(A @ x_gauss - f)

print('='*70)
print('МЕТОД ГАУССА С ВЫБОРОМ ГЛАВНОГО ЭЛЕМЕНТА')
print('='*70)
print('\nРешение:')
print(x_gauss)
print(f'\nНевязка ||Ax - f||: {residual_gauss:.6e}')
print('='*70)

---
## 2. Метод LU-разложения

### Описание метода:

Метод основан на разложении матрицы $A$ на произведение двух матриц:

$$A = LU$$

где:
- $L$ - нижняя треугольная матрица с единицами на диагонали
- $U$ - верхняя треугольная матрица

**Алгоритм:**

1. **LU-разложение** (вычисление элементов $L$ и $U$):

$$u_{ij} = a_{ij} - \sum_{k=1}^{i-1} l_{ik}u_{kj}, \quad j \geq i$$

$$l_{ij} = \frac{1}{u_{jj}}\left(a_{ij} - \sum_{k=1}^{j-1} l_{ik}u_{kj}\right), \quad i > j$$

2. **Решение $Ly = f$** (прямая подстановка):

$$y_i = f_i - \sum_{j=1}^{i-1} l_{ij}y_j$$

3. **Решение $Ux = y$** (обратная подстановка):

$$x_i = \frac{1}{u_{ii}}\left(y_i - \sum_{j=i+1}^{n} u_{ij}x_j\right)$$

### Преимущества:
- Разложение вычисляется один раз
- Эффективно для решения нескольких систем с одной матрицей

### Сложность: $O(n^3)$ для разложения, $O(n^2)$ для решения

In [ ]:
def lu_decomposition(A: np.ndarray, f: np.ndarray) -> np.ndarray:
    """Решение СЛАУ методом LU-разложения"""
    n = len(f)
    L = np.zeros((n, n))
    U = np.zeros((n, n))
    
    # LU-разложение
    for i in range(n):
        # Верхняя треугольная матрица U
        for j in range(i, n):
            sum_val = 0.0
            for k in range(i):
                sum_val += L[i, k] * U[k, j]
            U[i, j] = A[i, j] - sum_val
        
        # Нижняя треугольная матрица L
        L[i, i] = 1.0
        for j in range(i + 1, n):
            sum_val = 0.0
            for k in range(i):
                sum_val += L[j, k] * U[k, i]
            if U[i, i] != 0:
                L[j, i] = (A[j, i] - sum_val) / U[i, i]
    
    # Решение Ly = f (прямая подстановка)
    y = np.zeros(n)
    for i in range(n):
        sum_val = 0.0
        for j in range(i):
            sum_val += L[i, j] * y[j]
        y[i] = f[i] - sum_val
    
    # Решение Ux = y (обратная подстановка)
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        sum_val = 0.0
        for j in range(i + 1, n):
            sum_val += U[i, j] * x[j]
        x[i] = (y[i] - sum_val) / U[i, i]
    
    return x

# Решение
x_lu = lu_decomposition(A, f)
residual_lu = np.linalg.norm(A @ x_lu - f)

print('='*70)
print('МЕТОД LU-РАЗЛОЖЕНИЯ')
print('='*70)
print('\nРешение:')
print(x_lu)
print(f'\nНевязка ||Ax - f||: {residual_lu:.6e}')
print(f'\nСравнение с методом Гаусса:')
print(f'  ||x_LU - x_Gauss||: {np.linalg.norm(x_lu - x_gauss):.6e}')
print('='*70)

---
# ИТЕРАЦИОННЫЕ МЕТОДЫ

Итерационные методы строят последовательность приближений $x^{(0)}, x^{(1)}, x^{(2)}, \ldots$, сходящуюся к точному решению.

---
# Сводная таблица результатов

## Сравнение всех методов


| № | Метод | Тип | Невязка | Итерации |
|---|-------|-----|---------|----------|
| 1 | Гаусс с выбором главного элемента | Прямой | 5.192593e-17 | - |
| 2 | LU-разложение | Прямой | 6.655553e-17 | - |
| 3 | Якоби | Итерационный | 7.516293e-11 | 36 |
| 4 | Зейделя | Итерационный | 9.427840e-11 | 10 |
| 5 | SOR (ω=1.1) | Итерационный | 2.692848e-11 | 14 |
| 6 | Градиентный спуск | Итерационный | 8.259367e-09 | 16 |
| 7 | Минимальные невязки | Итерационный | 1.155556e-08 | 12 |
| 8 | Сопряжённые градиенты | Итерационный | 4.460448e-09 | 8 |
| 9 | BiCGSTAB | Итерационный | 1.872000e-12 | 5 |


---
# Графики убывания невязки для итерационных методов

## Классические итерационные методы

### 3. Метод Якоби

**Количество итераций:** 36

**Конечная невязка:** 7.516294e-11

![Метод Якоби](images/jacobi.png)

---
### 4. Метод Зейделя

**Количество итераций:** 10

**Конечная невязка:** 9.427842e-11

![Метод Зейделя](images/seidel.png)

---
### 5. Метод верхней релаксации (SOR)

**Количество итераций:** 14

**Конечная невязка:** 2.692837e-11

**Параметр релаксации:** ω = 1.1

![Метод SOR](images/sor.png)

---
## Градиентные методы

### 6. Метод градиентного спуска

**Количество итераций:** 16

**Конечная невязка:** 8.259367e-09

![Метод градиентного спуска](images/gradient_descent.png)

---
### 7. Метод минимальных невязок

**Количество итераций:** 12

**Конечная невязка:** 1.155556e-08

![Метод минимальных невязок](images/minimal_residual.png)

---
### 8. Метод сопряжённых градиентов

**Количество итераций:** 8

**Конечная невязка:** 4.369876e-09

*Примечание: для несимметричной матрицы используется преобразование A^T A x = A^T f*

![Метод сопряжённых градиентов](images/conjugate_gradient.png)

---
### 9. Стабилизированный метод бисопряжённых градиентов (BiCGSTAB)

**Количество итераций:** 5

**Конечная невязка:** 1.282844e-09

![Метод BiCGSTAB](images/bicgstab.png)

---
# Сравнительный анализ

## Сравнение классических итерационных методов

![Сравнение классических методов](images/comparison_classical.png)

### Выводы:
- Метод Зейделя сходится **~в 2 раза быстрее** метода Якоби
- Метод SOR с параметром ω=1.1 показывает **лучшую** скорость сходимости среди классических методов
- Якоби: 36 итераций
- Зейделя: 10 итераций
- SOR: 14 итераций

---
## Сравнение градиентных методов

![Сравнение градиентных методов](images/comparison_gradient.png)

### Выводы:
- Метод сопряжённых градиентов и BiCGSTAB показывают **наилучшую** скорость сходимости
- Метод минимальных невязок эффективнее простого градиентного спуска
- BiCGSTAB работает напрямую с несимметричной матрицей
- Градиентный спуск: 16 итераций
- Минимальные невязки: 12 итераций
- Сопряжённые градиенты: 8 итераций
- BiCGSTAB: 5 итераций

---
## Общее сравнение всех итерационных методов

![Сравнение всех методов](images/comparison_all.png)

---
# Выводы

##  Рекомендации по выбору метода:

| Ситуация | Рекомендуемый метод |
|----------|--------------------|
| Малые плотные системы (n < 100) | **Гаусс** или **LU** |
| Большие разреженные системы | **BiCGSTAB** или **Сопряжённые градиенты** |
| Симметричная положительно определённая | **Сопряжённые градиенты** |
| Несимметричная матрица | **BiCGSTAB** |
| Несколько систем с одной матрицей | **LU-разложение** |
| Параллельные вычисления | **Якоби** |
| Нужна максимальная скорость | **BiCGSTAB** или **SOR** |

---

##  Итоговый рейтинг по скорости сходимости:

1.  **BiCGSTAB** - 5 итераций
2.  **Сопряжённые градиенты** - 8 итераций
3.  **Минимальные невязки** - 12 итераций
4. **SOR** - 14 итераций
5. **Градиентный спуск** - 16 итераций
6. **Зейделя** - 10 итераций
7. **Якоби** - 36 итераций

Прямые методы (Гаусс, LU) дают решение за фиксированное число операций O(n³).